## Проект: RND Exploratory Behavior

**Команда: Сергей Червонцев, Иван Провилков**


Мы исследуем exploration поведение агента используя "Random Network Distillation (RND)" (https://arxiv.org/abs/1810.12894). 

Интересно узнать, как ведет себя агент не имея награды от среды, когда он пользуется только своим внутренним "интересом", который обеспечивается с помощью RND. Меньше ли получается итоговое качество, сколько времени занимает такое обучение и какие факторы на него влияют?

### Вначале мы поставили себе задачу написать агента и воспроизвести результаты статьи про RND на игре "Montezuma revenge".

### Описание среды:

Montezuma revenge это игра Atari, в которой агент должен пройти через множество комнат (всего их 99) в лабиринте, чтобы найти сокровище. Определенные комнаты закрыты дверями и чтобы туда попасть нужно найти и подобрать ключ. В комнатах есть монстры, которые убивают персонажа и их нужно избегать. Также персонаж умирает от падения с высоты или попадания в ловушку (падение в лаву). В комнатах есть различные алмазы, которые игрок может подбирать, а также снаряжение, которое можно надеть (факелы, мечи...). У персонажа есть 5 жизней, то есть он может 5 раз умереть до начала новой игры.

Состояния среды: RGB Картинка 210x160 пикселей (мы рескейлим до 84х84)

Пространство действий: агент может бегать в разные стороны и прыгать, всего 18 Discrete действий.

Extrinsic Reward: 


### Графики обучения:

У нас есть две основные модели. Первая учится с комбинированным ревардом как от среды, так и от случайной сети. Вторая модель учится только с ревардом от случайной сети.

Награду от среды мы считаем даже если она не участвует в обучении агента.

Обозначения в логах, на которые надо обратить внимание:

int_reward_per_episode: RND награда в зависимости от эпизода

int_reward_per_steps: RND награда в зависимости от шага (1 шаг -- одно обновление параметров)

num_steps: количество шагов от эпизода

reward_per_rollout: награда агента от среды в зависимости от шага

reward_per_episode: награда агента от среды в зависимости от эпизода 


Model with extrinsic + intrinsic reward:


Model with only intrinsic reward:



### Результаты базового запуска:

**После какого-то времени дебага и исправления возникающих проблем, мы смогли запустить агента с лоссом из статьи RND. Как видно из графиков обучения и видео (можно посмотреть на странице проекта гитхаба) агент с комбинированным ревардом научился заходить в 9 комнат.**

**Затем мы запустили агента только с intrinsic reward. Этот агент научился заходить в 3 комнаты, за то же время обучения.**

### Гиперпараметры:

Все гиперпараметры эксперимента можно посмотреть в config.yaml

В случае "only intrinsic" обучения ExtCoeff: 0.0, в случае обычного обучения ExtCoeff: 1.0

#### Немного о параметрах:

**NumWorkers: у нас реализовано многопоточное выполнение действий сразу NumWorkers агентами с одинаковыми весами для накопления статистик для обучения.**

**Rollout steps: количество шагов, которые проходит каждый агент для одного step оптимайзера**


In [26]:
!cat config.yaml

EnvName: MontezumaRevengeNoFrameskip-v4
MaxStepsPerEpisode: 4500
ImageHeight: 84
ImageWidth: 84
UseStickyAction: True
StickyActionProb: 0.25

NumWorkers: 128
IntrinsicRewardDiscount: 0.99
NumInitSteps: 1024

RolloutSteps: 128
ExtRewardDiscount: 0.999
IntRewardDiscount: 0.99
ExtCoeff: 0.0
IntCoeff: 1.0
LearningRate: 0.0001
ClipGradNorm: 10.0
BatchSize: 4096
EpochSteps: 4
SavePath: "/home/chervontsev/checkpoints/RndAgentFast.ckpt"

NumEpochs: 10000

RNDUpdateProportion: 0.125
PPOEntropyCoeff: 0.001
PPORewardEps: 0.1
PPOAdvLambda: 0.95
UseVTraceCorrection: False

UseTPU: False


### Для запуска модели нужно установить нужные библиотеки, мы используем python3.6

In [23]:
!pip install -r requirements.txt

### Запуск агента с обученными весами

1. Скачиваем веса из google drive: https://drive.google.com/drive/folders/15RBD-BrWUlylYLR13u7NWRhnMTAb9cWs?usp=sharing
2. Пишем путь до весов в config.yaml. 
Пример: SavePath: "/home/chervontsev/checkpoints/RndAgentFast.ckpt"
3. Запускаем eval.py скрипт: python eval.py
4. Видео игры агента появится в папке MontezumaRevengeNoFrameskip-v4_example_run в корне проекта.

In [24]:
!python eval.py

N_updates 10000
Finished, total reward is 0
All visited rooms: {1}


**После удачного запуска агентов, когда мы убедились, что алгоритм работает, мы решили поставить себе задачу ускорить процесс обучения, так как обучение на этой среде занимает очень много времени и у нас не очень много ресурсов.**

**Этот процесс мы разделили на несколько этапов:**

1. После профилирования мы обнаружили, что много времени (~10 секунд на 1 .accumulate_rollout_steps() со 128 лернерами(воркерами)) занимает подготовка статистик необходимых для шага обучения агента, и примерно столько-же времени занимает сам шаг обучения. То есть ГПУ простаивает половину времени.Чтобы решить эту проблему мы решили сделать раздельное обучение actor-а (который собирает статистики необходимые для работы PPO) и learner-а (который делает апдейты оптимайзером) на примере того как это сделано в IMPALA (Включая V-trace).
2. Подумали, что стоит переписать имеющийся код на TPU.

**Мы реализовали пункт 1, что дало нам ускорение примерно в 2.3 раза. Результаты находятся в ветке impala (https://github.com/JanRocketMan/lightning_rnd/tree/impala)**

**Второй пункт остается на future work**

# Результаты

**Нам удалось воспроизвести статью Random Network Distillation.**

**Наши эксперименты показали, что с помощью RND агент может учиться без награды от среды. Однако, это занимает больше времени, а также ему труднее попасть в новые комнаты, в которые он как-то попал в прошлый раз. (Так как нет сильного дополнительного реварда от среды).**

**Мы реализовали оптимизацию, которая разделяет обучение actor-а и learner-а, что дало нам ускорение в 2.3 раза.**